# ENF Extraction from Audio and Video Files

## Import Standard Modules

In [ ]:
import sys
import matplotlib.pyplot as plt
import numpy as np

!# Install the Python modules that are not yet present on Colab
try:
  import py7zr
except:
  !pip install py7zr
  import py7zr

## Load Custom ENF Modules from Github

In [ ]:
!# Clone the files on github to Colab so that they can be used
![ -d enf-matching ] || git clone https://github.com/CoRoe/enf-matching.git

# Add the path of the just cloned Python files to the Python path:
if not '/content/enf-matching' in sys.path:
    sys.path.insert(0, '/content/enf-matching')
#print(sys.path)

from enf import AudioClipEnf
from enf import notch_filter

In [ ]:
# @title Choose an audio or video file to analyse

# TODO: The current mechanism is akward. Check
# https://colab.research.google.com/github/NeuromatchAcademy/course-content-dl/blob/main/projects/ComputerVision/spectrogram_analysis.ipynb
# for ideas.

filename = "enf-matching/samplemedia/001.wav" # @param {"type":"string","placeholder":"Audio or video file"}

clip = AudioClipEnf()
if clip.loadAudioFile(filename):
  print(f"Loaded '{filename}' ok, sample rate {clip.sampleRate()}")
else:
  print(f"Failed to load audio file '{filename}'")

## Spectrogram

This step displays the spectrogram of the input file without any filtering. The brighter the colour the stronger the frequency component.

In [ ]:
NFFT = 2048
fig, (ax2) = plt.subplots(nrows=1, sharex=True)
Pxx, freqs, bins, im = ax2.specgram(clip.data, NFFT=NFFT, Fs=clip.sampleRate())
# The `specgram` method returns 4 objects. They are:
# - Pxx: the periodogram
# - freqs: the frequency vector
# - bins: the centers of the time bins
# - im: the .image.AxesImage instance representing the data in the plot
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Frequency (Hz)')
ax2.set_title('Spectrogram')

In [ ]:
# @title For the next steps, some parameters have to be chosen.
grid_freq = "50" # @param ["50","60"]
harmonic = "2" # @param ["1","2"]


# Spectrogram of Filtered Data

In [ ]:
filter_quality = 10 # Filter quality

# FIXME: Output of the notch filter is not plausible and does not agree
# with the STFT result.
filtered_data = notch_filter(clip.data, int(grid_freq), clip.sampleRate(),
                             filter_quality)
NFFT = 1024
fig, (ax2) = plt.subplots(nrows=1, sharex=True)
Pxx, freqs, bins, im = ax2.specgram(filtered_data, NFFT=NFFT, Fs=clip.sampleRate())
# The `specgram` method returns 4 objects. They are:
# - Pxx: the periodogram
# - freqs: the frequency vector
# - bins: the centers of the time bins
# - im: the .image.AxesImage instance representing the data in the plot
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Frequency (Hz)')
ax2.set_title('Spectrogram')

# ENF over Time

This step determines the variation of the ENF signal over time.

In [ ]:
clip.makeEnf(int(grid_freq), 0.200, int(harmonic))
t, f_enf = clip.getEnf()
fig, (ax1) = plt.subplots(nrows=1, sharex=True)
ax1.plot(t, f_enf/1000)
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('ENF (Hz)')